In [1]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

In [2]:
#query = "anmeldung"
#target_service = 120686

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(100)
eval.head()

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...


Funktionen

In [4]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df#.reset_index()
    rank = temp.index[temp["id"]==str(targetService)].values[0]
    return rank

Logging

In [5]:
class Logger:
    
    def __init__(self, chatbot, eval_df):
        self.liste = []
        self.chatbot = chatbot
        self.eval_df = eval_df
    
    def getDataFrame(self): # TODO
        pass
    
    def log(self, ID, t, row, answer = None): 
        target_service = row["documentId"]
        
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        
        
        if answer is not None:

            
            nResults = len(self.chatbot.df.index)
            question = self.chatbot.generateQuestion()
            answer = answer

            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)
        else:
            #nInitialResults = len(chatbot.df.index)
            #initialRank = np.nan
            row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [6]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(10)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()
chatbot = cb.Chatbot(solrhandler,clusterer,tpc_dterminator)

lg = Logger(chatbot,eval)   

for (ID, row) in eval.iterrows():#np.arange(0,len(eval.index)-1):#[0,1,2,3,4]
    
    # fun
    query = row["initialQuestion"]#"Ummeldung"
    target_service = row["documentId"]
    
    # Log
    
    # TODO ERROR???
    try:
        chatbot.initialQuery(query)
    except Exception as e:
        if(str(e) == 'response'):
            next
        elif(str(e) == 'no solr output'):
            next
    
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(ID,t, row, answer)
    if answer is None:
        continue
    
    
    while not chatbot.refineResultset(answer, True): # TODO REcluster
        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        if answer is None: 
            break
        lg.log(ID, t, row,answer)
    t+=1
    #lg.log(i,t,answer)

In [7]:
#query
#len(solrhandler.get_df_from_query(query))

In [8]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "service_name", "query", "question", "answer", "rank", "nResults"])
log_df["rank"] = log_df["rank"]+1
log_df.head()

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um zulassung?,True,3.0,7.0
1,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,None,None,NaN,NaN
2,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um ablegen?,False,4.0,10.0
3,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um sars?,True,4.0,6.0
4,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um hinweiscrn?,False,2.0,3.0


Statistics

In [9]:
# Rank Improvement
log_df["rank_improvement"] = log_df.groupby(["ID"])["rank"].diff()
log_df.groupby(["t"])["rank_improvement"].mean()

t
0         NaN
1   -0.647059
2   -1.166667
3   -1.000000
4   -1.000000
5    0.000000
Name: rank_improvement, dtype: float64

In [10]:
# nSuggestions Improvement
log_df["sugg_improvement"] = log_df.groupby(["ID"])["nResults"].diff()
log_df.groupby(["t"])["sugg_improvement"].mean()

t
0         NaN
1   -3.529412
2   -1.333333
3   -1.500000
4   -1.000000
5   -2.000000
Name: sugg_improvement, dtype: float64

In [11]:
# MRR
log_df["MRR"] = 1/log_df["rank"]

log_df.groupby(["t"])["nResults", "rank", "MRR" ].mean()

,nResults,rank,MRR
t,,,
0,7.647887,2.042254,0.718712
1,6.176471,1.705882,0.754902
2,5.500000,1.666667,0.722222
3,5.500000,1.500000,0.750000
4,5.000000,1.000000,1.000000
5,3.000000,1.000000,1.000000


In [ ]:
# Turns


In [14]:
#test = pd.merge(eval, log_df, on=["file", "dialogId"], how="inner")
#test

log_df.head()

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults,rank_improvement,sugg_improvement,MRR
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um zulassung?,True,3.0,7.0,NaN,NaN,0.333333
1,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,None,None,NaN,NaN,NaN,NaN,NaN
2,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um ablegen?,False,4.0,10.0,NaN,NaN,0.250000
3,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um sars?,True,4.0,6.0,0.0,-4.0,0.250000
4,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um hinweiscrn?,False,2.0,3.0,-2.0,-3.0,0.500000
